### CIE-Project B || Group - 7

#### Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy.signal import resample, find_peaks
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pickle
import time

# from sklearn import preprocessing ##importing for normalization
# from sklearn.decomposition import PCA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler, minmax_scale, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor

import tensorflow as tf
from tensorflow import Tensor
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, \
                         AveragePooling1D, GlobalAveragePooling1D, \
                         Add, BatchNormalization, ReLU, Input
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

# plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['legend.frameon'] = True
# plt.rcParams['figure.figsize'] = (8, 8)
%matplotlib widget
# %matplotlib inline

tf.random.set_seed(1)
np.random.seed(2)

##### Get the root directory and change to it

In [ ]:
rootdir = os.getcwd()
os.chdir(rootdir)

#### Folder Variables Assignment

In [ ]:
# Names of the folders
Training_Folder = 'EPOT_DATA'
Validation_Aug_Folder = 'Validation_augmented_data'
Experiment = 'Experiment_Data_Group7'
Ex_Validation = 'Experimental_validation'

counter = 0

#### Parsing

##### Parse training and validation-augmented data

In [ ]:
# Initialize lists for training and validation
training_tr = []; training_tr_labels = []

validation_aug = []; validation_aug_labels = [] # validation augmented data

#------------------------------------------------------------------------------

for subdir, dirs, files in os.walk(rootdir):
    
    if Training_Folder.lower() in subdir.lower():
        for file in files:
            if file.endswith('.mat'):
                lbl = [file.split('_')[1], file.split('_')[2][:3]]
                lb = np.asarray(lbl, dtype=float)
                # load mat-file
                mat = loadmat(os.path.join(subdir, file))
                # get the numpy data from the mat file
                mat = mat['num_data']
                # append data to the list
                training_tr.append(mat)
                training_tr_labels.append(lb)

    elif Validation_Aug_Folder.lower() in subdir.lower():
        for file in files:
            if file.endswith('.mat'):
                lbl = [file.split('_')[1], file.split('_')[2][:3]]
                lb = np.asarray(lbl, dtype=float)
                # load mat-file
                mat = loadmat(os.path.join(subdir, file))
                # get the numpy data from the mat file
                mat = mat['num_data']
                # append data to the list
                validation_aug.append(mat)
                validation_aug_labels.append(lb)

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)

l = 6

fig.suptitle(f'EPOT Training Data - {training_tr_labels[l]}')

axs[0,0].plot(training_tr[l][:,0], training_tr[l][:,1], label='P - 1', c = 'r')
axs[0,1].plot(training_tr[l][:,0], training_tr[l][:,2], label='P - 2', c = 'g')
axs[1,0].plot(training_tr[l][:,0], training_tr[l][:,3], label='P - 3', c = 'b')
axs[1,1].plot(training_tr[l][:,0], training_tr[l][:,4], label='P - 4', c = 'y')

fig.legend()
plt.show()

##### Parse, Cut, Normalize, and Resample the experimental data

In [ ]:
# Initialize lists for training and validation
ex_data = []; ex_labels = []

#------------------------------------------------------------------------------

for subdir, dirs, files in os.walk(rootdir):
    
    if Experiment.lower() in subdir.lower():
        for file in files:
            if file.endswith('.txt'):
                
                with open(os.path.join(subdir, file), 'r') as f:
                    x = pd.read_csv(os.path.join(subdir, file), delimiter='\t', decimal=',', skiprows=8, header=None)
                    x = np.asarray(x, dtype=float)
                    ex_data.append(x)

In [ ]:
plt.close()
l = 5
plt.plot(ex_data[l][:,0], ex_data[l][:,1], label='P - 1', c = 'r')
plt.show()

In [ ]:
ex_cut_data = []
for i in range(len(ex_data)):
    
    threshold = 0.1

    x = np.where(abs(ex_data[i][:,1])>threshold)[0][0]
    y = np.where(abs(ex_data[i][:,2])>threshold)[0][0]
    z = np.where(abs(ex_data[i][:,3])>threshold)[0][0]
    p = np.where(abs(ex_data[i][:,4])>threshold)[0][0]

    start = min(x,y,z,p)
    end = start+200

    d = np.empty_like(ex_data[i][start:end,:])

    d[:,0] = ex_data[i][start:end, 0]
    d[:,1] = (-1)*ex_data[i][start:end, 1]
    d[:,2] = (-1)*ex_data[i][start:end, 2]
    d[:,3] = (-1)*ex_data[i][start:end, 4]
    d[:,4] = (-1)*ex_data[i][start:end, 3]

    ex_cut_data.append(d)

In [ ]:
plt.close()
l = 5
plt.plot(ex_cut_data[l][:,0], ex_cut_data[l][:,1], label='P - 1')
plt.plot(ex_cut_data[l][:,0], ex_cut_data[l][:,2], label='P - 2')
plt.plot(ex_cut_data[l][:,0], ex_cut_data[l][:,3], label='P - 3')
plt.plot(ex_cut_data[l][:,0], ex_cut_data[l][:,4], label='P - 4')
plt.show()

Normalize and Resample experimental data

In [ ]:
# create variables for normalized data
ex_norm = np.zeros_like(np.asarray(ex_cut_data))

for i in range (0, len(ex_cut_data)):
    for j in range (1, 5):
        ex_norm[i][:,j] = minmax_scale(ex_cut_data[i][:,j])

In [ ]:
ex_resample = []

# create a dataframe from the normalized data
for i in range (0, len(ex_norm)):
    ex_resample.append(pd.DataFrame(ex_norm[i], columns = ['time', '1', '2', '3', '4']))
    # drop the time column
    ex_resample[i] = ex_resample[i].drop(columns = ['time'])

size = 100

for i in range (0, len(ex_resample)):
    ex_resample[i] = resample(ex_resample[i], size)

np.asarray(ex_resample).shape

##### Parse, Cut, Normalize, and Resample the experimental validation data

In [ ]:
# Initialize lists for training and validation
ex_val_data = []; ex_val_labels = []

#------------------------------------------------------------------------------

for subdir, dirs, files in os.walk(rootdir):
    
    if Ex_Validation.lower() in subdir.lower():
        for file in files:
            if file.endswith('.txt'):
                
                with open(os.path.join(subdir, file), 'r') as f:
                    # print(file.split('_')[2], file.split('_')[3], file.split('_')[5][:3])
                    lbl = [file.split('_')[2], file.split('_')[3], file.split('_')[5][:3]]
                    lb = np.asarray(lbl, dtype=float)

                    x = pd.read_csv(os.path.join(subdir, file), delimiter='\t', decimal=',', skiprows=8, header=None)
                    x = np.asarray(x, dtype=float)
                    ex_val_data.append(x)
                    ex_val_labels.append(lb)


ex_val_trails = np.asarray(ex_val_labels)[:,2]
ex_val_labels = np.delete(ex_val_labels, 2, 1)

In [ ]:
# ex_val_cut_data=[]
# for i in range(0,len(ex_val_data)):
#     data=ex_data[i][:,0]
    
#     start_time = 0.04996
#     end_time = 0.05016
#     start=np.where(data==(start_time))
#     end = np.where(data==(end_time))
    
#     d = np.zeros(((start[0][0]+200-start[0][0]),5))

#     d[:,0]=ex_val_data[i][start[0][0]:start[0][0]+200,0]
#     d[:,1]=ex_val_data[i][start[0][0]:start[0][0]+200,1]
#     d[:,2]=ex_val_data[i][start[0][0]:start[0][0]+200,2]
#     d[:,3]=ex_val_data[i][start[0][0]:start[0][0]+200,4]
#     d[:,4]=ex_val_data[i][start[0][0]:start[0][0]+200,3]
    
#     ex_val_cut_data.append(d)
#----------------------------------------------------------------------------------------


ex_val_cut_data = []
for i in range(len(ex_val_data)):
    
    threshold = 0.1

    x = np.where(abs(ex_val_data[i][:,1])>threshold)[0][0]
    y = np.where(abs(ex_val_data[i][:,2])>threshold)[0][0]
    z = np.where(abs(ex_val_data[i][:,3])>threshold)[0][0]
    p = np.where(abs(ex_val_data[i][:,4])>threshold)[0][0]

    start = min(x,y,z,p)
    end = start+180

    d = np.empty_like(ex_val_data[i][start:end,:])

    d[:,0] = ex_val_data[i][start:end, 0]
    d[:,1] = (-1)*ex_val_data[i][start:end, 1]
    d[:,2] = (-1)*ex_val_data[i][start:end, 2]
    d[:,3] = (-1)*ex_val_data[i][start:end, 4]
    d[:,4] = (-1)*ex_val_data[i][start:end, 3]

    ex_val_cut_data.append(d)

Normalize and Resample experimental validation data

In [ ]:
# create variables for normalized data
ex_val_norm = np.zeros_like(np.asarray(ex_val_cut_data)); ex_val_norm_labels = np.zeros_like(np.asarray(ex_val_cut_data))

for i in range (0, len(ex_val_cut_data)):
    for j in range (1, 5):
        ex_val_norm[i][:,j] = minmax_scale(ex_val_cut_data[i][:,j])

scalar = MinMaxScaler()
ex_val_norm_labels = scalar.fit_transform(ex_val_labels)

In [ ]:
ex_val_resample = []

# create a dataframe from the normalized data
for i in range (0, len(ex_val_norm)):
    ex_val_resample.append(pd.DataFrame(ex_val_norm[i], columns = ['time', '1', '2', '3', '4']))
    # drop the time column
    ex_val_resample[i] = ex_val_resample[i].drop(columns = ['time'])

size = 100

for i in range (0, len(ex_val_resample)):
    ex_val_resample[i] = resample(ex_val_resample[i], size)

np.asarray(ex_val_resample).shape

#### Augmentation

##### Variable initialization

In [ ]:
## -----------------------------------------------------------------------------
#Assuming P1 on top and counter clockwise order
## -----------------------------------------------------------------------------


# Variables for the augmented data
training_tl = np.zeros_like(np.asarray(training_tr)); training_tl_labels = np.zeros_like(np.asarray(training_tr_labels))
training_bl = np.zeros_like(np.asarray(training_tr)); training_bl_labels = np.zeros_like(np.asarray(training_tr_labels))
training_br = np.zeros_like(np.asarray(training_tr)); training_br_labels = np.zeros_like(np.asarray(training_tr_labels))

##### Top-Left

In [ ]:
for i in range (0, len(training_tr)):
        training_tl[i][:,0] = training_tr[i][:,0]
        training_tl[i][:,1] = training_tr[i][:,1]
        training_tl[i][:,2] = training_tr[i][:,4]
        training_tl[i][:,3] = training_tr[i][:,3]
        training_tl[i][:,4] = training_tr[i][:,2]

for i in range (0, len(training_tl_labels)):
    training_tl_labels[i][0] = 500 - training_tr_labels[i][0]
    training_tl_labels[i][1] = training_tr_labels[i][1]

##### Bottom-Left

In [ ]:
for i in range (0, len(training_bl)):
    training_bl[i][:,0] = training_tl[i][:,0]
    training_bl[i][:,1] = training_tl[i][:,3]
    training_bl[i][:,2] = training_tl[i][:,2]
    training_bl[i][:,3] = training_tl[i][:,1]
    training_bl[i][:,4] = training_tl[i][:,4]

for i in range (0, len(training_bl_labels)):
    training_bl_labels[i][0] = 500. - training_tr_labels[i][0]
    training_bl_labels[i][1] = 500. - training_tr_labels[i][1]

##### Bottom-Right

In [ ]:
for i in range (0, len(training_br)):
        training_br[i][:,0] = training_bl[i][:,0]
        training_br[i][:,1] = training_bl[i][:,1]
        training_br[i][:,2] = training_bl[i][:,4]
        training_br[i][:,3] = training_bl[i][:,3]
        training_br[i][:,4] = training_bl[i][:,2]

for i in range (0, len(training_br_labels)):
    training_br_labels[i][0] = training_tr_labels[i][0]
    training_br_labels[i][1] = 500 - training_tr_labels[i][1]

##### Delete overlapping data points

In [ ]:
training_tl = np.delete(np.asarray(training_tl), np.where(np.asanyarray(training_tl_labels) == 250.)[0], axis = 0)
training_tl_labels = np.delete(np.asarray(training_tl_labels), np.where(np.asanyarray(training_tl_labels) == 250.)[0], axis = 0)

training_bl = np.delete(np.asarray(training_bl), 0, axis = 0)
training_bl_labels = np.delete(np.asarray(training_bl_labels), 0, axis = 0)

training_br = np.delete(np.asarray(training_br), np.where(np.asanyarray(training_br_labels) == 250.)[0], axis = 0)
training_br_labels = np.delete(np.asarray(training_br_labels), np.where(np.asanyarray(training_br_labels) == 250.)[0], axis = 0)

##### Makesure the augmentation is done on all quadrants

In [ ]:
print(f'{np.asarray(training_tl).shape} \t{np.asarray(training_tr).shape} \n{np.asarray(training_bl).shape} \t{np.asarray(training_br).shape} \n{counter}')

In [ ]:
plt.close()
for i in range(0, len(training_tr_labels)):
    plt.plot(training_tr_labels[i][0], training_tr_labels[i][1], 'o', c = 'r')
for j in range (0, len(training_tl_labels)):
    plt.plot(training_tl_labels[j][0], training_tl_labels[j][1], 's', c = 'g')
    plt.plot(training_br_labels[j][0], training_br_labels[j][1], 'P', c = 'y')
for k in range (0, len(training_bl_labels)):
    plt.plot(training_bl_labels[k][0], training_bl_labels[k][1], '*', c = 'b')

plt.xticks(np.arange(175, 330, 5), rotation = 'vertical')
plt.yticks(np.arange(175, 330, 5))
plt.show()

#### Concatenate, Cut, Normalize, Resample Training Data

##### Concatenate

In [ ]:
training = np.concatenate((training_tr, training_tl, training_bl, training_br), axis = 0)
training_labels = np.concatenate((training_tr_labels, training_tl_labels, training_bl_labels, training_br_labels), axis = 0)

print(f'Shape of all training set: {training.shape} \nShape of all training labels: {training_labels.shape}')

##### check with the validation augmented data

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 4, sharex=True)

val_c = 1 # Given validation folder has 4 data points. so use - (0, 1, 2, 3)

# find out the index of the validation set in the training data
x = []
for i in range (0, len(training_labels)):
    if (training_labels[i][0] == validation_aug_labels[val_c][0]) and (training_labels[i][1] == validation_aug_labels[val_c][1]):
        x.append(i)

# plot to compare the validation set with the training set
fig.suptitle(f'EPOT Training Data Coordinates - {training_labels[x[0]]} \nValidation Augmented Data Coordinates - {validation_aug_labels[val_c]}')

axs[0,0].plot(training[x[0]][:,0], training[x[0]][:,1], label='P - 1', c = 'r')
axs[0,1].plot(training[x[0]][:,0], training[x[0]][:,2], label='P - 2', c = 'g')
axs[0,2].plot(training[x[0]][:,0], training[x[0]][:,3], label='P - 3', c = 'b')
axs[0,3].plot(training[x[0]][:,0], training[x[0]][:,4], label='P - 4', c = 'y')

axs[1,0].plot(validation_aug[val_c][:,0], validation_aug[val_c][:,1], label='P - 1', c = 'r')
axs[1,1].plot(validation_aug[val_c][:,0], validation_aug[val_c][:,2], label='P - 2', c = 'g')
axs[1,2].plot(validation_aug[val_c][:,0], validation_aug[val_c][:,3], label='P - 3', c = 'b')
axs[1,3].plot(validation_aug[val_c][:,0], validation_aug[val_c][:,4], label='P - 4', c = 'y')

# check if the data is exactly the same
for i in range (1, 5):
    if np.all(training[x[0]][:,i] == validation_aug[val_c][:,i], axis = 0):
        print('Data is augmented correctly')

fig.legend()
plt.show()

In [ ]:
diff = training[x[0]][:,1] - validation_aug[val_c][:,1]
diff

##### -- Check -- for outliers

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)

for i in range(len(training)):
    axs[0,0].plot(training[i][:,0], training[i][:,1], label='Training P-1')
    axs[0,1].plot(training[i][:,0], training[i][:,2], label='Training P-2')
    axs[1,0].plot(training[i][:,0], training[i][:,3], label='Training P-3')
    axs[1,1].plot(training[i][:,0], training[i][:,4], label='Training P-4')

plt.show()


In [ ]:
%matplotlib inline
sns.set(rc={'figure.figsize':(30, 30)})

x = []; y = []; z = []

for i in range(len(training)):
    xi = training_labels[i][0]
    x.append(xi)
    yi = training_labels[i][1]
    y.append(yi)
    zi = pd.DataFrame(training[i][:,1]).max()[0]
    # zi = pd.DataFrame(training[i][:,2]).max()[0]
    # zi = pd.DataFrame(training[i][:,3]).max()[0]
    # zi = pd.DataFrame(training[i][:,4]).max()[0]

    z.append(zi)

df = pd.DataFrame({'x': x, 'y': y, 'z': z})

heatmap1_data = pd.pivot_table(df, values='z', index=['y'], columns='x')

ax = sns.heatmap(heatmap1_data, 
                square=True, 
                annot = True, 
                annot_kws={"fontsize": 10}, 
                fmt='.1f', 
                linewidths=0.1, 
                linecolor='gray',
                cbar_kws = dict(use_gridspec=False, location='right'))

ax.invert_yaxis()

Experimental Validation

In [ ]:

%matplotlib inline
sns.set(rc={'figure.figsize':(30, 30)})

x = []; y = []; z = []

for i in range(len(ex_val_data)):
    xi = ex_val_labels[i][0]
    x.append(xi)
    yi = ex_val_labels[i][1]
    y.append(yi)
    zi = pd.DataFrame(ex_val_data[i][:,1]).max()[0]
    # zi = pd.DataFrame(training[i][:,2]).max()[0]
    # zi = pd.DataFrame(training[i][:,3]).max()[0]
    # zi = pd.DataFrame(training[i][:,4]).max()[0]

    z.append(zi)

df = pd.DataFrame({'x': x, 'y': y, 'z': z})

heatmap1_data = pd.pivot_table(df, values='z', index=['y'], columns='x')

ax = sns.heatmap(heatmap1_data, 
                square=True, 
                annot = True, 
                annot_kws={"fontsize": 10}, 
                fmt='.1f', 
                linewidths=0.1, 
                linecolor='gray',
                cbar_kws = dict(use_gridspec=False, location='right'))

ax.invert_yaxis()


##### Cutting Training Data

In [ ]:
train_cut = []
for i in range(len(training)):
    
    threshold = 0.1

    x = np.where(abs(training[i][:,1])>threshold)[0][0]
    y = np.where(abs(training[i][:,2])>threshold)[0][0]
    z = np.where(abs(training[i][:,3])>threshold)[0][0]
    p = np.where(abs(training[i][:,4])>threshold)[0][0]

    start = min(x,y,z,p)

    d = np.empty_like(training[i][start:,:])

    d[:,0] = training[i][start:, 0]
    d[:,1] = training[i][start:, 1]
    d[:,2] = training[i][start:, 2]
    d[:,3] = training[i][start:, 3]
    d[:,4] = training[i][start:, 4]

    train_cut.append(d)

##### Normalize Training Data

In [ ]:
# maxim = []
# minim = []
# for i in range (len(train_cut)):
#     for j in range (1,5):
#         p = train_cut[i][:,j].max()
#         q = train_cut[i][:,j].min()

#         maxim.append(p)
#         minim.append(q)
# # print(np.asarray(maxim).max(),np.asarray(minim).min())

# create variables for normalized data
# train_norm = np.zeros_like(np.array(train_cut)) 
train_norm = []
train_norm_labels = np.zeros_like(np.asarray(training_labels))

for i in range (0, len(train_cut)):
    
    n = np.zeros_like(train_cut[i])
    n[:,1] = minmax_scale(train_cut[i][:,1])
    n[:,2] = minmax_scale(train_cut[i][:,2])
    n[:,3] = minmax_scale(train_cut[i][:,3])
    n[:,4] = minmax_scale(train_cut[i][:,4])
    
    train_norm.append(n)

    
scalar = MinMaxScaler()
train_norm_labels = scalar.fit_transform(training_labels)

In [ ]:
maxim = []
minim = []
for i in range (len(train_norm)):
    for j in range (1,5):
        p = train_norm[i][:,j].max()
        q = train_norm[i][:,j].min()

        maxim.append(p)
        minim.append(q)
print(np.asarray(maxim).max(),np.asarray(minim).min())

In [ ]:
plt.close()
plt.plot(train_norm_labels[:,0], train_norm_labels[:,1], 'o', c = 'r', label = 'Normalized Training Data Labels')
plt.legend(loc='lower center', bbox_to_anchor=(1, -0.5))
plt.show()

##### Resample Training Data

In [ ]:
train_resample = []

# create a dataframe from the normalized data
for i in range (0, len(training)):
    train_resample.append(pd.DataFrame(train_norm[i], columns = ['time', '1', '2', '3', '4']))
    # drop the time column
    train_resample[i] = train_resample[i].drop(columns = ['time'])

size = 100

for i in range (0, len(training)):
    train_resample[i] = resample(train_resample[i], size)

np.asarray(train_resample).shape

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 1)

axs[0].plot(training[10][:,2], c = 'r', label = 'Original Training Data')
axs[1].plot(train_resample[10][:,1], c = 'b', label = 'cut/ norm/ resampled Data')

axs[0].legend(loc='lower left')
axs[1].legend(loc='lower left')
plt.show()

#### Compare Experimental, Experimental-Validation and Training Data

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 4, sharey=True)

ex_val_c = 20 # Given validation folder has 24 data points. so use - (0, 1, .... , 23)

# find out the index of the validation set in the training data
x = []
for i in range (0, len(training_labels)):
    if (training_labels[i][0] == ex_val_labels[ex_val_c][0]) and (training_labels[i][1] == ex_val_labels[ex_val_c][1]):
        x.append(i)

# plot to compare the validation set with the training set
fig.suptitle(f'Training Data Coordinates - {training_labels[x[0]]} \nExperimental Validation Data Coordinates - {ex_val_labels[ex_val_c]} - Trial [{ex_val_trails[ex_val_c]}]')

axs[0,0].plot(training[x[0]][:,0], training[x[0]][:,1], label='P - 1', c = 'r')
axs[0,1].plot(training[x[0]][:,0], training[x[0]][:,2], label='P - 2', c = 'g')
axs[0,2].plot(training[x[0]][:,0], training[x[0]][:,3], label='P - 3', c = 'b')
axs[0,3].plot(training[x[0]][:,0], training[x[0]][:,4], label='P - 4', c = 'y')

axs[1,0].plot(ex_val_cut_data[ex_val_c][:,0], ex_val_cut_data[ex_val_c][:,1], label='P - 1', c = 'r')
axs[1,1].plot(ex_val_cut_data[ex_val_c][:,0], ex_val_cut_data[ex_val_c][:,2], label='P - 2', c = 'g')
axs[1,2].plot(ex_val_cut_data[ex_val_c][:,0], ex_val_cut_data[ex_val_c][:,3], label='P - 3', c = 'b')
axs[1,3].plot(ex_val_cut_data[ex_val_c][:,0], ex_val_cut_data[ex_val_c][:,4], label='P - 4', c = 'y')

# fig.legend()
plt.show()

In [ ]:
plt.close()
fig, axs = plt.subplots(1, 4, sharex=True, sharey=True)

for i in range(4):
    axs[0].plot(ex_val_cut_data[i][:,0], ex_val_cut_data[i][:,1], c = 'r', label = 'Exp Val Data - P-1')
    axs[1].plot(ex_val_cut_data[i][:,0], ex_val_cut_data[i][:,2], c = 'g', label = 'Exp Val Data - P-2')
    axs[2].plot(ex_val_cut_data[i][:,0], ex_val_cut_data[i][:,3], c = 'b', label = 'Exp Val Data - P-3')
    axs[3].plot(ex_val_cut_data[i][:,0], ex_val_cut_data[i][:,4], c = 'y', label = 'Exp Val Data - P-4')

plt.show()

#### Export data to pickle

In [ ]:
# with open('training_data.pickle', 'wb') as f:
#     pickle.dump(training, f)

# with open('train_resample.pickle', 'wb') as f:
#     pickle.dump(train_resample, f)
# with open('train_norm_labels.pickle', 'wb') as f:
#     pickle.dump(train_norm_labels, f)

# with open('training_labels.pickle', 'wb') as f:
#     pickle.dump(training_labels, f)

# with open('exp_validation_data.pickle', 'wb') as f:
#     pickle.dump(ex_val_data, f)
# with open('exp_validation_labels.pickle', 'wb') as f:
#     pickle.dump(ex_val_labels, f)
# with open('exp_validation_trails.pickle', 'wb') as f:
#     pickle.dump(ex_val_trails, f)


# with open('exp_val_resample.pickle', 'wb') as f:
#     pickle.dump(ex_val_resample, f)
# with open('exp_val_norm_labels.pickle', 'wb') as f:
#     pickle.dump(ex_val_norm_labels, f)

# with open('ex_resample.pickle', 'wb') as f:
#     pickle.dump(ex_resample, f)

#### AI Model Tuning and Training

##### Split the data into training and testing sets

In [ ]:
# Extract the data from the pickle files
'''
with open('train_resample.pickle', 'rb') as f:
    train_resample = pickle.load(f)
with open('train_norm_labels.pickle', 'rb') as f:
    train_norm_labels = pickle.load(f)
'''

X = np.asarray(train_resample)
y = np.asarray(train_norm_labels)

# create split data from the normalized data
size = 0.2
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=size)
print(f'Length of training data - {len(x_train)} length of labels - {len(y_train)} \nLength of test data - {len(x_test)} length of labels - {len(y_test)}')

##### TensorBoard

In [ ]:
# %tensorboard --logdir=logs/

##### Model Training and Tuning

DNN Tuning

In [ ]:

log_files = f'DNN_log.{int(time.time())}'
tuner_logs= f'tunelog/DNN_tuner_log.{int(time.time())}'

tensorboard = TensorBoard(log_dir=f'logs/{log_files}', 
                          histogram_freq=1, 
                          write_graph=True,
                          write_grads=True)


def dnnModel(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(100,4)))
    model.add(Dense(units=hp.Int('layer_1_nodes', 
                    min_value=32, max_value=512, step=32), 
                    activation=hp.Choice("activation_1", ["relu", "sigmoid", "tanh"])))
    
    for i in range(hp.Int('n_layers', min_value=0, max_value=4, step=1)):
        model.add(Dense(units=hp.Int(f'layer_{i+1}_nodes', min_value=32, max_value=512, step=32), activation=hp.Choice(f"activation_{i+1}", ["relu", "sigmoid", "tanh"])))
    
    model.add(Dense(units=2, activation=hp.Choice("activation_out", ['sigmoid', 'linear']))) # output layer

    model.summary()

    # Compile model
    # lr = hp.Float("lr", min_value=1e-4, max_value=1e-2)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    sgd = tf.keras.optimizers.SGD(learning_rate=0.0001)

    model.compile(loss = hp.Choice('loss', ['mae']),
                optimizer = hp.Choice('optimizer', ['Adam', 'RMSprop']), 
                metrics=['Accuracy'])
    
    return model

# Hyperparameter tuning
tuner = RandomSearch(dnnModel,
                    objective='val_Accuracy',
                    max_trials=50,
                    executions_per_trial=2,
                    directory=tuner_logs,
                    project_name='DNN_tuning')

tuner.search(x=x_train,
            y=y_train,
            epochs=60,
            batch_size=32,
            validation_data=(x_test, y_test),
            callbacks=[tensorboard])
            '''

In [ ]:
# bestModels = tuner.get_best_models(num_models=2)

In [ ]:
# Save the keras model
# model.save(f'model/complete_tunedModel_97perAcc.{int(time.time())}.h5')

CNN Tuning

In [ ]:

log_files = f'CNN_log.{int(time.time())}'
tuner_logs= f'tunelog/CNN_tuner_log.{int(time.time())}'

tensorboard = TensorBoard(log_dir=f'logs/{log_files}', 
                          histogram_freq=1, 
                          write_graph=True,
                          write_grads=True)

def cnnModel(hp):
    model = Sequential()

    model.add(Conv1D(filters=hp.Int('conv_1_filter', min_value=32, max_value=512, step=32), 
                     kernel_size=hp.Int('Kernal_1', min_value=2, max_value=5, step=1), 
                     strides=hp.Int('stride_1', min_value=1, max_value=5, step=1),
                     activation=hp.Choice("activation_1", ["relu", "sigmoid"]), 
                     input_shape=(100,4)))
    
    model.add(MaxPooling1D(pool_size=hp.Int('pool_1', min_value=2, max_value=5, step=1))),

    for i in range(hp.Int('n_layers', min_value=0, max_value=4, step=1)):
        model.add(Conv1D(filters=hp.Int(f'conv_{i+1}_filter', min_value=32, max_value=512, step=32), 
                         kernel_size=hp.Int(f'Kernal_{i+1}', min_value=2, max_value=5, step=1), 
                         strides=hp.Int(f'stride_{i+1}', min_value=1, max_value=5, step=1),
                         activation=hp.Choice(f"activation_{i+1}", ["relu", "sigmoid"])))
        model.add(MaxPooling1D(pool_size=hp.Int(f'pool_{i+1}', min_value=2, max_value=5, step=1)))

    model.add(Flatten())


    model.add(Dense(units=hp.Int('FC_layer_1_nodes',
                    min_value=32, max_value=512, step=32),
                    activation=hp.Choice("FC_activation_1", ["relu", "sigmoid"])))
    
    model.add(Dense(units=hp.Int('FC_layer_2_nodes',
                    min_value=32, max_value=512, step=32),
                    activation=hp.Choice("FC_activation_2", ["relu", "sigmoid"])))

    model.add(Dense(units=2, activation=hp.Choice("activation_out", ['sigmoid', 'linear']))) # output layer

    model.summary()

    # Compile model
    # lr = hp.Float("lr", min_value=1e-4, max_value=1e-2)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    sgd = tf.keras.optimizers.SGD(learning_rate=0.0001)

    model.compile(loss = hp.Choice('loss', ['mse', 'mae']),
                optimizer = hp.Choice('optimizer', ['Adam', 'RMSprop']), 
                metrics=['Accuracy'])

    return model

# Hyperparameter tuning
tuner = RandomSearch(cnnModel,
                    objective='val_Accuracy',
                    max_trials=560,
                    executions_per_trial=3,
                    directory=tuner_logs,
                    project_name='CNN_tuning')

tuner.search(x=x_train,
            y=y_train,
            epochs=60,
            batch_size=32,
            validation_data=(x_test, y_test),
            callbacks=[tensorboard])


In [ ]:
# bestModels = tuner.get_best_models(num_models=2)

#### FFNN Model

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(100,4)))

model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=96, activation='relu'))
model.add(Dense(units=12, activation='relu'))

model.add(Dense(units=2, activation='linear'))

model.summary()

# Compile model
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss = 'mse', optimizer = 'Adam', metrics=['Accuracy'])

# Fit the model
history = model.fit(x_train, y_train, epochs=50, verbose=1, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
# model.save(f'model/DNN_val_acc_97perAcc.{int(time.time())}.h5')

In [ ]:
mean_squared_error(scalar.inverse_transform(y_test), scalar.inverse_transform(model.predict(x_test)), squared=True)

In [ ]:
# mean_squared_error((ex_val_labels), scalar.inverse_transform(model.predict(ex_val_resample)), squared=True)

In [ ]:
plt.close()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['Accuracy'], label='Training Accuracy')
plt.plot(history.history['val_Accuracy'], label='Validation Accuracy')

plt.yticks(np.arange(0, 1, 0.05))

plt.tight_layout()
plt.legend()
plt.show()

#### mAlexNet

In [ ]:
Alex4=Sequential()

Alex4.add(Conv1D(filters=96, kernel_size=5, strides=4, activation='relu', input_shape=(100,4), padding='same'))
Alex4.add(MaxPooling1D(pool_size=3, strides= 2, padding='valid'))
Alex4.add(Dropout(0.3))

Alex4.add(Conv1D(filters=256, strides=1, kernel_size=5, activation='relu', padding='same'))
Alex4.add(MaxPooling1D(pool_size=3, strides=2, padding='valid'))

Alex4.add(Conv1D(filters=384, strides=1, kernel_size=3, activation='relu', padding='same'))
Alex4.add(Dropout(0.4))

Alex4.add(Conv1D(filters=256, strides=1, kernel_size=3, activation='relu', padding='same'))

Alex4.add(Conv1D(filters=256, strides=1, kernel_size=3, activation='relu', padding='same'))
Alex4.add(MaxPooling1D(pool_size=3, strides=2, padding='valid'))

# Fully Connected Layers
Alex4.add(Flatten())
Alex4.add(Dense(units=128, activation='relu',use_bias='False'))
Alex4.add(Dense(units=64, activation='relu',use_bias='False'))
Alex4.add(Dense(units=2, activation='linear',use_bias='False'))
Alex4.summary()

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

Alex4.compile(loss = 'mse', optimizer = 'Adam', metrics=['Accuracy'])

# Fit the cnn_model
Alex_history = Alex4.fit(x_train, y_train, epochs=90, verbose=1, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
mean_squared_error(scalar.inverse_transform(y_test), scalar.inverse_transform(Alex4.predict(x_test)), squared=True)
# mean_absolute_error(scalar.inverse_transform(y_test), scalar.inverse_transform(Alex4.predict(x_test)))

In [ ]:
# Alex4.save(f'model/FINAL_CNN_val_acc_97_16_MSE.{int(time.time())}.h5')

In [ ]:
plt.close()
plt.plot(Alex_history.history['loss'], label='Training Loss')
plt.plot(Alex_history.history['val_loss'], label='Validation Loss')
plt.plot(Alex_history.history['Accuracy'], label='Training Accuracy')
plt.plot(Alex_history.history['val_Accuracy'], label='Validation Accuracy')

plt.yticks(np.arange(0, 1, 0.05))

plt.tight_layout()
plt.xlabel('Epochs')
plt.legend()
plt.show()

#### Extension Other Networks

##### ResNet

In [ ]:
def relu_bn(inputs: Tensor) -> Tensor:
    bn = BatchNormalization()(inputs)
    relu = ReLU()(bn)
    return relu

def CNN_block(x: Tensor, filters: int, kernel_size: int = 3) -> Tensor:
    # i = BatchNormalization()(x)
    j = Conv1D(kernel_size=3,
            strides=1,
            filters=filters,
            padding="same")(x)
    return j

def residual_block(x: Tensor, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv1D(kernel_size=kernel_size,
               strides= 1,
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv1D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)
    y = BatchNormalization()(y)

    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(100,4))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv1D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
        
    num_blocks_list = [2, 1, 2]
    t = CNN_block(t, filters = 128)
    for j in range(num_blocks_list[0]):
        t = residual_block(t, filters=128)

    t = Dropout(0.3)(t)

    t = CNN_block(t, filters = 64)
    for j in range(num_blocks_list[1]):
        t = residual_block(t, filters=64)
    
    t = Dropout(0.3)(t)

    t = CNN_block(t, filters = 64)
    for j in range(num_blocks_list[2]):
        t = residual_block(t, filters=64)
    
    t = AveragePooling1D(pool_size = 3)(t)
    t = Flatten()(t)
    # t = Dense(units=128, activation='relu')(t)
    t = Dense(units=32, activation="relu")(t)
    outputs = Dense(2, activation='linear')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='Adam',
        loss='mse',
        metrics=['Accuracy']
    )

    return model

rnet1 = create_res_net() # or create_plain_net()
rnet1.summary()

In [ ]:
# timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# name = 'cifar-10_res_net_30-'+timestr # or 'cifar-10_plain_net_30-'+timestr

# checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# os.system('mkdir {}'.format(checkpoint_dir))

# # save model after each epoch
# cp_callback = ModelCheckpoint(
#     filepath=checkpoint_path,
#     verbose=1
# )
# tensorboard_callback = TensorBoard(
#     log_dir='tensorboard_logs/'+name,
#     histogram_freq=1
# )

rNet1_history = rnet1.fit(
    x=x_train,
    y=y_train,
    epochs=90,
    verbose=1,
    validation_data=(x_test, y_test),
    batch_size=32,
    # callbacks=[cp_callback, tensorboard_callback]
)

In [ ]:
mean_squared_error(scalar.inverse_transform(y_test), scalar.inverse_transform(rnet1.predict(x_test)), squared=True)
# mean_absolute_error(scalar.inverse_transform(y_test), scalar.inverse_transform(rnet.predict(x_test)))

In [ ]:
plt.close()
plt.plot(rNet1_history.history['loss'], label='Training Loss')
plt.plot(rNet1_history.history['val_loss'], label='Validation Loss')
plt.plot(rNet1_history.history['Accuracy'], label='Training Accuracy')
plt.plot(rNet1_history.history['val_Accuracy'], label='Validation Accuracy')

plt.yticks(np.arange(0, 1, 0.05))
plt.ylim(bottom=0, top=1)
plt.tight_layout()
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
# rnet.save('ResNet-60MSE.h5')

##### VGG

In [ ]:
VGG2 = Sequential()

VGG2.add(Conv1D(filters=64, kernel_size=3, strides=2,activation='relu', input_shape=(100,4), padding='same'))
VGG2.add(Conv1D(filters=64, kernel_size=3, strides=1,activation='relu',padding='valid'))
VGG2.add(Conv1D(filters=64, kernel_size=1, strides=1,activation='relu' ))

VGG2.add(MaxPooling1D(pool_size=2, strides=2))

VGG2.add(Conv1D(filters=128, kernel_size=3, strides=1,activation='relu',padding='valid'))
VGG2.add(Conv1D(filters=128, kernel_size=1, strides=1,activation='relu' ))

VGG2.add(MaxPooling1D(pool_size=2, strides=2))

VGG2.add(Conv1D(filters=256, kernel_size=3, strides=1,activation='relu',padding='valid'))
VGG2.add(Conv1D(filters=256, kernel_size=1, strides=1,activation='relu' ))

VGG2.add(MaxPooling1D(pool_size=2, strides=2))

VGG2.add(Conv1D(filters=256, kernel_size=3, strides=1,activation='relu',padding='valid'))
VGG2.add(Conv1D(filters=384, kernel_size=1, strides=1,activation='relu'))
VGG2.add(Conv1D(filters=100, kernel_size=1, strides=1,activation='relu'))
VGG2.add(MaxPooling1D(pool_size=2, strides=2))

VGG2.add(Flatten())

VGG2.add(Dense(units=2,activation='linear'))

VGG2.summary()

VGG2.compile(loss = 'mse', optimizer = 'Adam', metrics=['Accuracy'])
VGG2_act=VGG2.fit(x_train, y_train, epochs=40, verbose=1, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
mean_squared_error(scalar.inverse_transform(y_test), scalar.inverse_transform(VGG2.predict(x_test)), squared=True)

In [ ]:
plt.close()
plt.plot(VGG2_act.history['loss'], label='Training Loss')
plt.plot(VGG2_act.history['val_loss'], label='Validation Loss')
plt.plot(VGG2_act.history['Accuracy'], label='Training Accuracy')
plt.plot(VGG2_act.history['val_Accuracy'], label='Validation Accuracy')

plt.yticks(np.arange(0, 1, 0.05))
plt.ylim(0, 1)
plt.tight_layout()
plt.xlabel('Epochs')
plt.legend()
plt.show()

#### Predict experimental validation data

In [ ]:
# model = tf.keras.models.load_model('model/tunedModel_98perAcc.1640573960.h5')
# model = tf.keras.models.load_model('Alex3.h5')
# model = tf.keras.models.load_model('model/CNN_val_acc_96_48_MSEperAcc.1642513870.h5')
# model.summary()

In [ ]:
y_pred = Alex4.predict(np.asarray(ex_val_resample))
y_abs = ex_val_labels

y_pred = scalar.inverse_transform(np.asarray(y_pred))
diff = y_abs-y_pred
for i in range(len(y_pred)):
    print(f'Absolute Trial {ex_val_trails[i]}- {y_abs[i]} Predicted - {y_pred[i]} \tDifference - {y_abs[i] - y_pred[i]}')

In [ ]:
mean_squared_error(y_abs, y_pred, squared=True)
mean_absolute_error(y_abs, y_pred)

#### Experimental Group - 7 Data (and post-processing)

##### Prediction

In [ ]:
training_labels.shape

In [ ]:
y_pred_g7 = Alex4.predict(np.asarray(ex_resample))
y_pred_g7 = scalar.inverse_transform(y_pred_g7)
y_pred_g7

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 4, sharey=True)

ex_c = 1 # Given validation folder has 18 data points. so use - (0, 1, .... , 17)

# find out the index of the validation set in the training data
p = []

for i in range (len(y_pred_g7)):
    xi = np.where(training_labels[:,0] == min(training_labels[:,0], key=lambda x:abs(x-y_pred_g7[i][0])))[0]
    yi = np.where(training_labels[:,1] == min(training_labels[:,1], key=lambda x:abs(x-y_pred_g7[i][1])))[0]
    pi = np.intersect1d(xi, yi)[0]
    
    p.append(pi)

# plot to compare the validation set with the training set
fig.suptitle(f'Closest Training Data Coordinates - {training_labels[p[ex_c]]} \nExperimental Predicted Coordinates - {y_pred_g7[ex_c]}')

axs[0,0].plot(training[p[ex_c]][:,1], label='P - 1', c = 'r')
axs[0,1].plot(training[p[ex_c]][:,2], label='P - 2', c = 'g')
axs[0,2].plot(training[p[ex_c]][:,3], label='P - 3', c = 'b')
axs[0,3].plot(training[p[ex_c]][:,4], label='P - 4', c = 'y')

axs[1,0].plot(ex_cut_data[ex_c][:,1], label='P - 1', c = 'r')
axs[1,1].plot(ex_cut_data[ex_c][:,2], label='P - 2', c = 'g')
axs[1,2].plot(ex_cut_data[ex_c][:,3], label='P - 3', c = 'b')
axs[1,3].plot(ex_cut_data[ex_c][:,4], label='P - 4', c = 'y')

# fig.legend()
plt.show()

##### Post processing of mAlexNet model

sorting the exact experimental data

In [ ]:
'''
Group_7	AA	250	265
Group_7	AA	245	255
Group_7	AA	235	235
Group_7	AA	265	235
Group_7	AA	250	250
Group_7	AA	255	265
Group_7	B	225	200
Group_7	B	270	300
Group_7	B	200	225
Group_7	B	235	300
Group_7	B	260	320
Group_7	B	300	285
Group_7	A	250	265
Group_7	A	245	255
Group_7	A	235	235
Group_7	A	265	235
Group_7	A	250	250
Group_7	A	265	255
'''

exact_ex_labels = np.array(
[[250,	265], 
[245,	255],
[235,	235],
[265,	235],
[250,	250],
[255,	265],
[225,	200],
[270,	300],
[200,	225],
[235,	300],
[260,	320],
[300,	285],
[250,	265],
[245,	255],
[235,	235],
[265,	235],
[250,	250],
[265,	255]])

In [ ]:
# predictions made from AlexNet model
pred_ex_labels = np.array(
[[253.32326, 244.85556],
[234.51955, 233.11015],
[243.45721, 259.58542],
[260.85577, 225.5717 ],
[241.64229, 252.67258],
[262.7103 , 247.9715 ],
[236.75409, 299.41733],
[230.32304, 228.36177],
[264.8639 , 298.7226 ],
[252.88496, 258.21362],
[260.63187, 232.50989],
[252.09583, 252.05911],
[201.56601, 229.92183],
[256.91928, 257.9066 ],
[245.16226, 252.61618],
[224.14635, 201.8915 ],
[269.60422, 310.06744],
[298.5857 , 281.49817]])

# pred_ex_labels = scalar.inverse_transform(rnet1.predict(np.asarray(ex_resample)))

In [ ]:
print(exact_ex_labels.shape, pred_ex_labels.shape)

In [ ]:
plt.close()
plt.plot(training_labels[:,0], training_labels[:,1], '.', color = 'g', label='Training Data', alpha = 0.3)
plt.plot(pred_ex_labels[:,0], pred_ex_labels[:,1], 'o', color = 'b', label='Predicted')
plt.plot(exact_ex_labels[:,0], exact_ex_labels[:,1], 'o', color = 'r', label='Exact')
plt.scatter(exact_ex_labels[:,0], exact_ex_labels[:,1], s=750, facecolors='none', edgecolors='k')

plt.xticks(np.arange(175, 330, 5), rotation = 'vertical')
plt.yticks(np.arange(175, 330, 5))

legend_x = 0.5
legend_y = -0.27
plt.legend(bbox_to_anchor=(legend_x, legend_y), loc = 'lower center', ncol = 3)
plt.show()

In [ ]:
x = []
p_loc=[]
for i in range(len(exact_ex_labels)):
    t = []
    for j in range(len(pred_ex_labels)):
        xi = (np.asarray(np.abs(exact_ex_labels[i]-pred_ex_labels[j])).sum())
        t.append(xi)
    p_loc.append(np.argmin(t))
    x.append(np.min(t))

In [ ]:
# reorganising the predicted labels
pred_ex_labels_org = np.empty_like(pred_ex_labels)
for i in range (len(p_loc)):
    pred_ex_labels_org[i] = pred_ex_labels[p_loc[i]]

print('Exact Coordinates \tPredicted Coordinates \t     Difference')
for i in range(len(p_loc)):
    print(f'{exact_ex_labels[i]}---------------{pred_ex_labels[p_loc[i]]}--------{exact_ex_labels[i] - pred_ex_labels[p_loc[i]]}')

print(f'MSE of the Group-7 data - {mean_squared_error(exact_ex_labels, pred_ex_labels_org)}')
print(f'MAE of the Group-7 data - {mean_absolute_error(exact_ex_labels, pred_ex_labels_org)}')

Reinforce prediction with post-processing

In [ ]:
# temp = 1

def find_leastmse_train(temp):
    # find out the index of the validation set in the training data
    p = []

    for i in range (len(pred_ex_labels)):
        xi = np.where(training_labels[:,0] == min(training_labels[:,0], key=lambda x:abs(x-pred_ex_labels[i][0])))[0]
        yi = np.where(training_labels[:,1] == min(training_labels[:,1], key=lambda x:abs(x-pred_ex_labels[i][1])))[0]
        pi = np.intersect1d(xi, yi)[0]
        
        p.append(pi)

    # for i in range(len(p)):
    #     print(f'{training_labels[p][i]} \t {pred_ex_labels[i]}')

    x_offset = [-5, 0, 5]
    y_offset = [-5, 0, 5]
    loc=[]; x=[]; y=[]

    for k in (x_offset):
        for l in (y_offset):
            xi = np.where(training_labels[:,0][p][temp] + k == training_labels[:,0])
            yi = np.where(training_labels[:,1][p][temp] + l == training_labels[:,1])
            x.append(xi)
            y.append(yi)
        
        for m in range(len(x)):
            for n in range(len(y)):
                loc.append(np.intersect1d(x[m], y[n])[0])
    locs = np.unique(loc)

    mse = []
    for i in range(len(locs)):
        err = mean_squared_error(train_resample[locs[i]], ex_resample[temp])
        mse.append(err)
    min_loc = locs[np.argmin(mse)]

    return locs, min_loc



leastmse_pred = []; leastmse_label = []
l = []; locs = []

for temp in range(len(pred_ex_labels)):
    try:    
        l, x = find_leastmse_train(temp)
        locs.append(l)
        leastmse_label.append(x)
        leastmse_pred.append(training_labels[x])
    except:
        print(f'{temp} - {pred_ex_labels[temp]} surrounding points are not in the training data')
        pass
    
    # print(f'loop - {temp}')

leastmse_pred = np.asarray(leastmse_pred)
leastmse_label = np.asarray(leastmse_label)
locs = np.asarray(locs)

leastmse_pred = np.insert(leastmse_pred, 3, values=[260, 225], axis=0)
leastmse_label = np.insert(leastmse_label, 3, values=722)
locs = np.insert(locs, 3, values=722, axis = 0)
print(np.asarray(leastmse_pred))

In [ ]:
plt.close()
fig, axs = plt.subplots(2, 4)
#      |
ex_c = 4 # <----- give the group-7 experimental label
tr_loc = leastmse_label[ex_c] # Given validation folder has 18 data points. so use - (0, 1, .... , 17)

# plot to compare the validation set with the training set
fig.suptitle(f'Least MSE Training Data Coordinates - {training_labels[tr_loc]}mm \nExp G7 Predicted Coordinates - {pred_ex_labels[ex_c]}mm')

axs[0,0].plot(train_resample[tr_loc][:,0], label='P - 1', c = 'r')
axs[0,1].plot(train_resample[tr_loc][:,1], label='P - 2', c = 'g')
axs[0,2].plot(train_resample[tr_loc][:,2], label='P - 3', c = 'b')
axs[0,3].plot(train_resample[tr_loc][:,3], label='P - 4', c = 'y')

axs[1,0].plot(ex_resample[ex_c][:,0], label='P - 1', c = 'r')
axs[1,1].plot(ex_resample[ex_c][:,1], label='P - 2', c = 'g')
axs[1,2].plot(ex_resample[ex_c][:,2], label='P - 3', c = 'b')
axs[1,3].plot(ex_resample[ex_c][:,3], label='P - 4', c = 'y')

# fig.legend()
plt.show()

In [ ]:
point = 4

plt.close()
plt.title('Find the least MSE Training Data Coordinate')

plt.plot(training_labels[:,0], training_labels[:,1], '.', color = 'g', label='Training Data', alpha = 0.3)
plt.plot(pred_ex_labels[:,0][point], pred_ex_labels[:,1][point], 's', color = 'k', label=f'Predicted Point {np.around(pred_ex_labels[point], decimals=1)}')

plt.scatter(training_labels[:,0][locs[point]], training_labels[:,1][locs[point]], c='b', alpha = 0.5, label = 'Buffer Set')
plt.scatter(training_labels[:,0][leastmse_label[point]], training_labels[:,1][leastmse_label[point]], c = 'r', label = f'Least MSE Point {training_labels[leastmse_label[point]]}')

plt.xticks(np.arange(175, 330, 5), rotation = 'vertical')
plt.yticks(np.arange(175, 330, 5))

plt.legend(bbox_to_anchor=(0.5, 0), loc = 'lower center', ncol = 2)
plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.show()

#### SVR Model

In [ ]:
X = train_resample
y = training_labels

# create split data from the normalized data
size = 0.2
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=size)
print(f'Length of training data - {len(x_train)} length of labels - {len(y_train)} \nLength of test data - {len(x_test)} and length of labels - {len(y_test)}')

x_train_flat = []
for i in range(len(x_train)):    
    x_dummy = np.asarray(x_train[i]).flatten()
    x_train_flat.append(x_dummy)
    
x_train_flat = np.asarray(x_train_flat)

x_test_flat = []
for i in range(len(x_test)):    
    x_dummy = np.asarray(x_test[i]).flatten()
    x_test_flat.append(x_dummy)
    
x_test_flat = np.asarray(x_test_flat)

print(np.asarray(x_train_flat).shape, np.asarray(y_train).shape)

In [ ]:
regr_multirf = MultiOutputRegressor(svm.SVR(kernel='rbf', C=1e5, gamma=0.03, epsilon=0.2, tol=1e-6))
regr_multirf.fit(x_train_flat, y_train)

y_pred = regr_multirf.predict(x_test_flat)
y_test-y_pred

In [ ]:
mean_squared_error(y_test, y_pred, squared=True)

In [ ]:
y_pred_exp = regr_multirf.predict(exp_val_X_flat)

mean_squared_error(exp_val_y,y_pred_exp)

In [ ]:
ex_X_flat = []
for i in range(len(ex_resample)):
    x_dummy = np.asarray(ex_resample[i]).flatten()
    ex_X_flat.append(x_dummy)

ex_X_flat = np.asarray(ex_X_flat)
ex_X_flat.shape

In [ ]:
y_pred_g7_knn = regr_multirf.predict(ex_X_flat)
y_pred_g7_knn

#### KNN Model

In [ ]:
X = train_resample
y = training_labels

exp_val_X = ex_val_resample
exp_val_y = ex_val_labels

print(np.asarray(X).shape, np.asarray(y).shape)

# create split data from the normalized data
size = 0.1
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=size)
print(f'Length of training data - {np.asarray(x_train).shape} length of labels - {np.asarray(y_train).shape} \nLength of test data - {np.asarray(x_test).shape} and length of labels - {np.asarray(y_test).shape}')

In [ ]:
x_train_flat = []
for i in range(len(x_train)):    
    x_dummy = np.asarray(x_train[i]).flatten()
    x_train_flat.append(x_dummy)
    
x_train_flat = np.asarray(x_train_flat)

x_test_flat = []
for i in range(len(x_test)):    
    x_dummy = np.asarray(x_test[i]).flatten()
    x_test_flat.append(x_dummy)
    
x_test_flat = np.asarray(x_test_flat)

exp_val_X_flat = []
for i in range(len(exp_val_X)):
    x_dummy = np.asarray(exp_val_X[i]).flatten()
    exp_val_X_flat.append(x_dummy)

exp_val_X_flat = np.asarray(exp_val_X_flat)

print(np.asarray(x_train_flat).shape, np.asarray(y_train).shape)
print(x_test_flat.shape, y_test.shape)
print(exp_val_X_flat.shape, exp_val_y.shape)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5, weights='distance')
knn.fit(x_train_flat, y_train)
y_pred = knn.predict(x_test_flat)
y_test-y_pred

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
y_pred_exp = knn.predict(exp_val_X_flat)

mean_squared_error(exp_val_y,y_pred_exp)

In [ ]:
ex_X_flat = []
for i in range(len(ex_resample)):
    x_dummy = np.asarray(ex_resample[i]).flatten()
    ex_X_flat.append(x_dummy)

ex_X_flat = np.asarray(ex_X_flat)
ex_X_flat.shape

In [ ]:
y_pred_g7_knn = knn.predict(ex_X_flat)
y_pred_g7_knn

#### Plots for report

In [ ]:
with open('_____/rNet1_history.pickle', 'rb') as f:
    rNet1_history = pickle.load(f)
with open('_____/wrNet_history.pickle', 'rb') as f:
    wrNet_history = pickle.load(f)
with open('_____/vgg_history.pickle', 'rb') as f:
    vgg_history = pickle.load(f)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (11, 7)
plt.close()

fig, ax  = plt.subplots(2, 2, sharey=True)

ax[0, 0].plot(history.history['Accuracy'], label = 'Training Accuracy')
ax[0, 0].plot(history.history['val_Accuracy'], label = 'Validation Accuracy')
ax[0, 0].plot(history.history['loss'], label = 'Loss')
ax[0, 0].plot(history.history['val_loss'], label = 'Validation Loss')
ax[0,0].set_title('FFNN Model')
# ax[0, 0].set_xlabel('Epochs')
ax[0, 0].set_ylabel('Accuracy/ Loss')

ax[0, 1].plot(Alex_history.history['Accuracy'])
ax[0, 1].plot(Alex_history.history['val_Accuracy'])
ax[0, 1].plot(Alex_history.history['loss'])
ax[0, 1].plot(Alex_history.history['val_loss'])
ax[0, 1].set_title('mAlexNet Model')
# ax[0, 1].set_xlabel('Epochs')
ax[0, 1].set_ylabel('Accuracy/ Loss')

ax[1, 0].plot(rNet1_history.history['Accuracy'])
ax[1, 0].plot(rNet1_history.history['val_Accuracy'])
ax[1, 0].plot(rNet1_history.history['loss'])
ax[1, 0].plot(rNet1_history.history['val_loss'])
ax[1, 0].set_title('ResNet Model')
ax[1, 0].set_xlabel('Epochs')
ax[1, 0].set_ylabel('Accuracy/ Loss')

ax[1, 1].plot(vgg_history['Accuracy'])
ax[1, 1].plot(vgg_history['val_Accuracy'])
ax[1, 1].plot(vgg_history['loss'])
ax[1, 1].plot(vgg_history['val_loss'])
ax[1, 1].set_title('VGG Model')
ax[1, 1].set_xlabel('Epochs')
ax[1, 1].set_ylabel('Accuracy/ Loss')

fig.legend(bbox_to_anchor=(0.25, 0), loc=2, ncol=2)
plt.yticks(np.arange(0, 1, step=0.1))
plt.xlabel('Epochs')
plt.ylim(0, 1)
plt.show()